In [4]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
import glob

In [8]:
#creating the feature labels
header = 'filename chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [12]:
#writing all rows and corresponding features to a csv file
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header) 
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
for genre in genres:
    sound_files = glob.glob(genre+"/*.au")
    for aud in sound_files:
        values, sr = librosa.load(aud)
        chroma_stft = librosa.feature.chroma_stft(y=values, sr=sr)
        rmse = librosa.feature.rms(y=values)
        spec_cent = librosa.feature.spectral_centroid(y=values, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=values, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=values, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(values)
        mfcc = librosa.feature.mfcc(y=values, sr=sr)
        to_append = f'{f} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}' 
        for val in mfcc:
            to_append += f' {np.mean(val)}'
        to_append += f' {genre}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [13]:
df = pd.read_csv('data.csv')
df.head()

,filename,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues\blues.00000.au,0.350129,0.130184,1784.122641,2002.412407,3805.723030,0.083045,-113.598824,121.570671,-19.162262,...,8.802088,-3.669941,5.744676,-5.162783,0.751707,-1.687854,-0.408730,-2.302677,1.222467,blues
1,blues\blues.00001.au,0.340849,0.095908,1530.261767,2038.987608,3550.713616,0.056040,-207.523834,123.985138,8.947019,...,5.377876,-2.234492,4.220704,-6.010687,0.929294,-0.728403,0.296872,-0.282430,0.530644,blues
2,blues\blues.00002.au,0.363538,0.175473,1552.832481,1747.754087,3042.410115,0.076291,-90.757164,140.440872,-29.084547,...,5.791247,-8.901966,-1.080160,-9.226244,2.448305,-7.724840,-1.819024,-3.440458,-2.238128,blues
3,blues\blues.00003.au,0.404854,0.141040,1070.153418,1596.422565,2184.879029,0.033309,-199.575134,150.086105,5.663404,...,6.078081,-2.478445,-1.081587,-2.874454,0.773994,-3.324069,0.636311,-0.615968,-3.405046,blues
4,blues\blues.00004.au,0.308526,0.091501,1835.128513,1748.410759,3579.957471,0.101461,-160.354172,126.209480,-35.581394,...,-2.810499,-6.934471,-7.553875,-9.167669,-4.515863,-5.451786,-0.915952,-4.408018,-11.704385,blues


In [15]:
data = df.drop(['filename'],axis=1)
data.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.350129,0.130184,1784.122641,2002.412407,3805.723030,0.083045,-113.598824,121.570671,-19.162262,42.363941,...,8.802088,-3.669941,5.744676,-5.162783,0.751707,-1.687854,-0.408730,-2.302677,1.222467,blues
1,0.340849,0.095908,1530.261767,2038.987608,3550.713616,0.056040,-207.523834,123.985138,8.947019,35.867149,...,5.377876,-2.234492,4.220704,-6.010687,0.929294,-0.728403,0.296872,-0.282430,0.530644,blues
2,0.363538,0.175473,1552.832481,1747.754087,3042.410115,0.076291,-90.757164,140.440872,-29.084547,31.686691,...,5.791247,-8.901966,-1.080160,-9.226244,2.448305,-7.724840,-1.819024,-3.440458,-2.238128,blues
3,0.404854,0.141040,1070.153418,1596.422565,2184.879029,0.033309,-199.575134,150.086105,5.663404,26.855282,...,6.078081,-2.478445,-1.081587,-2.874454,0.773994,-3.324069,0.636311,-0.615968,-3.405046,blues
4,0.308526,0.091501,1835.128513,1748.410759,3579.957471,0.101461,-160.354172,126.209480,-35.581394,22.139256,...,-2.810499,-6.934471,-7.553875,-9.167669,-4.515863,-5.451786,-0.915952,-4.408018,-11.704385,blues


In [16]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
trueval = encoder.fit_transform(genre_list)

In [17]:
scaler = StandardScaler()
inputs = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [18]:
x_train, x_test, y_train, y_test = train_test_split(intputs, trueval, test_size=0.2)

In [19]:
from keras import models
from keras import layers

In [24]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=128)

test_loss, test_acc = model.evaluate(x_test,y_test)
print('test_acc: ',test_acc)

Epoch 1/40
7/7 [==============================] - 0s 667us/step - loss: 2.1750 - accuracy: 0.2350
Epoch 2/40
7/7 [==============================] - 0s 667us/step - loss: 1.8563 - accuracy: 0.4062
Epoch 3/40
7/7 [==============================] - 0s 834us/step - loss: 1.6110 - accuracy: 0.4487
Epoch 4/40
7/7 [==============================] - 0s 667us/step - loss: 1.4293 - accuracy: 0.4963
Epoch 5/40
7/7 [==============================] - 0s 667us/step - loss: 1.2884 - accuracy: 0.5725
Epoch 6/40
7/7 [==============================] - 0s 667us/step - loss: 1.1699 - accuracy: 0.5900
Epoch 7/40
7/7 [==============================] - 0s 834us/step - loss: 1.0816 - accuracy: 0.6525
Epoch 8/40
7/7 [==============================] - 0s 834us/step - loss: 0.9986 - accuracy: 0.6800
Epoch 9/40
7/7 [==============================] - 0s 667us/step - loss: 0.9421 - accuracy: 0.6975
Epoch 10/40
7/7 [==============================] - 0s 667us/step - loss: 0.8845 - accuracy: 0.7163
Epoch 11/40
7/7 [==